In [1]:
# %config Completer.use_jedi = False

- [x] why is the count value when basic sorting is done different from gsheets wala value 
- the reasoning is while pivoting melting and lots of rows were lost 
- [ ] TODO-Add error testing to make sure the columns are in the exact same order
- [x] The above row is from the first method of trying there is a variance in the number of rows a huge variance
- the reasoning is while pivoting melting and lots of rows were lost 
- [ ] TODO-Based on Freq eff date and disc date make a counter as unit tests to check data is correct
- [ ] TODO-the progrmamming needs to be converted from a procedural to functional programmming paradigm
-[ ] TODO - performace hit on adding columns invidually in sliced dataframe versus to total dataframe
- [ ] text comment a134phase 6e2015 01 feb is a edge case for single flight per day per rotation 
- [ ] TODO : each set of textcomment and day needs to tested for testcomment set data is compliant 
- df2._is_view
- df._is_copy
-[ ] assigning the deptype column the pandas way rather the numpy way 


In [2]:
%%time
import datetime
import pickle
import re
import gspread
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



gc = gspread.service_account(filename='./crudg.json')
sheet = gc.open("flightsched").get_worksheet(3)
dataframe = pd.DataFrame(sheet.get_all_records())


Wall time: 4.75 s


In [3]:
# Set ipython's max row display
pd.set_option('display.max_row', 50)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)

In [4]:
dataframe.head()

,A/C No,Flt Desg,Freq,Dept Arp,Dept Time,Arvl Arp,Arrv Time,Equip,Block Time,Eff Date,Dis Date
0,1,149,6,HYD,10:10,IXB,12:30,321,2:20,1-May-21,1-May-21
1,1,619,6,IXB,13:05,HYD,15:20,321,2:15,1-May-21,1-May-21
2,1,6009,6,HYD,16:15,GOI,17:35,321,1:20,1-May-21,1-May-21
3,1,6645,6,GOI,18:05,HYD,19:25,321,1:20,1-May-21,1-May-21
4,1,149,7,HYD,10:10,IXB,12:30,321,2:20,2-May-21,2-May-21


In [4]:
%%time
def formatTidyData(paramDataframe):
    '''
    input: dataframe 
    
    transform: take each row of input dataframe / make each row of output for each valid date between eff and             discontinue date 

    output dataframe 
    '''
    Row_list = paramDataframe.values.tolist()
    masterList = []
    for xray in Row_list:
        startDate = xray[9] + "-" + xray[4]
        startDate = pd.to_datetime(startDate,
                                   format='%d-%m-%y-%h:%M',
                                   infer_datetime_format=True,
                                   utc=False)
        endDate = xray[10] + "-" + xray[4]
        endDate = pd.to_datetime(endDate,
                                 format='%d-%m-%y-%h:%M',
                                 infer_datetime_format=True,
                                 utc=False)
        #TODO : refactor startDate and Date can be directly given as strings
        requiredRange = pd.date_range(startDate, endDate, freq='D').tolist()
        daysList = str(xray[2]).replace(".", "")

        for alpha in requiredRange:
            # weekday starts at 0 hence 1 is added
            if str(alpha.weekday() + 1) in daysList:
                # TODO : refactor use .replace method to set required minutes and hours
                depDate = str(alpha.date()) + "-" + xray[4]
                depDate = pd.to_datetime(depDate,
                                         format='%d-%m-%y-%h:%M',
                                         infer_datetime_format=True,
                                         utc=False)
                arrDate = str(alpha.date()) + "-" + xray[6]
                arrDate = pd.to_datetime(arrDate,
                                         format='%d-%m-%y-%h:%M',
                                         infer_datetime_format=True,
                                         utc=False)
                masterList.append([
                    xray[0], xray[1], xray[3], xray[5], xray[7], xray[8],
                    xray[11], depDate, arrDate
                ])
    return pd.DataFrame(masterList,
                        columns=[
                            "Text_Comment", "Flt_Desg", "Dept_Arp", "Arvl_Arp",
                            "Subfleet", "Block_Time", "Service_Type",
                            "aocs_dep", "aocs_arr"
                        ])


#Wall time: 34.8 s
# 44.8 s ± 4.3 s per loop (mean ± std. dev. of 7 runs, 1 loop each) with time it
# 41.6 s ± 6.39 s per loop (mean ± std. dev. of 7 runs, 1 loop each)

Wall time: 0 ns


In [5]:
%%time
testpd = formatTidyData(dataframe)

Wall time: 37.7 s


- a manual step of doing it one month at a time for performance reasons 

In [6]:
testpd = testpd.loc[testpd["aocs_dep"].dt.month == 4]
testpd = testpd.copy()

In [7]:
# testpd = testpd.head(300)

In [8]:
%%time
# list(array of all days in our data )
uniqueDates = np.sort(testpd["aocs_dep"].dt.date.unique())
#list of all unique airports
aiportList = np.sort(testpd["Dept_Arp"].unique())

# list of Comment List which are unique
textCommentList = testpd["Text_Comment"].unique()
#list of all unique flightNumbers
flightNumberUniqueList = np.sort(testpd["Flt_Desg"].unique())

# adding column "engg_arr" and "deptype"
testpd["engg_arr"] = np.nan
testpd["engg_dep"] = np.nan
testpd["deptype"] = np.nan
testpd["cal_date"] = testpd["aocs_dep"].dt.day

# making block time as time delta in python
testpd["Block_Time"] = testpd["Block_Time"].astype("str")
splitString = testpd["Block_Time"].str.split(':', expand=True).astype(int)
testpd["Block_Time"] = pd.to_timedelta(
    splitString[0], unit='h') + pd.to_timedelta(splitString[1], unit='m')

# Adding datatypes of columns for performance improbement
testpd = testpd.astype({
    "deptype": "string",
    "engg_arr": "datetime64[ns]",
    "engg_dep": "datetime64[ns]",
    "Flt_Desg": "string",
    "Block_Time": "timedelta64[ns]"
})

# reorder columns
testpd = testpd[[
    'Dept_Arp', 'engg_arr', 'Flt_Desg', 'Text_Comment', 'Subfleet', 'Arvl_Arp',
    'aocs_dep', 'engg_dep', 'aocs_arr', 'deptype', 'Block_Time', 'Service_Type',
    'cal_date'
]]

# Index(['Dept Arp', 'engg_arr', 'Flt Desg', 'A/C No', 'Equip', 'Arvl Arp',
#        'engg_dep', 'aocs_arr', 'deptype', 'Block Time', 'cal_date'],
#       dtype='object')
# testpd.info()
# paramdf = paramdf[['Dept_Arp', 'engg_arr', 'Flt_Desg', 'Text_Comment', 'Subfleet','Arvl_Arp',  'aocs_dep','engg_dep', 'aocs_arr','deptype',  'Block_Time','Service_Type']]

Wall time: 533 ms


In [9]:
# %%cython --annotate
# import numpy as np


def addlastRowArrDep(paramdf):
    # temporary make columns consistent
    #  this is costing a lot on performance
    #  paramdf = paramdf[['Dept_Arp', 'engg_arr', 'Flt_Desg', 'Text_Comment', 'Subfleet','Arvl_Arp',  'aocs_dep','engg_dep', 'aocs_arr','deptype',  'Block_Time','Service_Type']]
    # edge case of only one departure for one text comment
    converted = paramdf.to_numpy()
    emptyarray = np.full((1, 13), np.nan)
    if (converted.shape[0] == 1):
        temparray = converted
        #  temparray[0][9]= "baseDep"
        temparray[0][7] = temparray[0][6]
        return temparray
    else:
        temparray = np.append(converted, emptyarray, axis=0)
        lentemparray = temparray.shape[0]
        temparray[lentemparray - 1][0:2] = temparray[lentemparray - 2][5:7]
        temparray[lentemparray - 1][2:5] = temparray[lentemparray - 2][2:5]
        for xray in range(0, lentemparray):
            if xray == 0:
                pass
                # TODO : this is not required and can be elimnated
                # onedf.at[xray,"deptype"] = "baseDep"
                # temparray[xray][9]= "baseDep"
                # onedf.at[xray,"engg_dep"] = onedf.at[xray,"aocs_dep"]
                # TODO : this is not required and can be elimnated
                # temparray[xray][7] = temparray[xray][6]
            elif xray == lentemparray - 1:
                pass
                # onedf.at[xray,"deptype"] = "baseArr"
                # temparray[xray][9]= "baseArr"
            else:
                # onedf.at[xray,"engg_arr"] = onedf.at[(xray-1),"aocs_arr"]
                temparray[xray][1] = temparray[xray - 1][8]
                # onedf.at[xray,"engg_dep"] = onedf.at[(xray),"aocs_dep"]
                # TODO : this is not required and can be elimnated
                # temparray[xray][7] = temparray[xray][6]
                # onedf.at[xray,"deptype"] = "Transit"
                # temparray[xray][9]= "Transit"
        return temparray

In [10]:
# %%time
# y_df =testpd.loc[(testpd["Text_Comment"]== textCommentList[0] ) & (testpd["aocs_dep"].dt.date==uniqueDates[3])]
# y_df= y_df.copy()
# y_df =  make_last_row(y_df)
# y_df = AddBaseArrDep(y_df)
# y_df = y_df[['Dept_Arp', 'engg_arr', 'Flt_Desg', 'Text_Comment', 'Subfleet','Arvl_Arp',  'aocs_dep','engg_dep', 'aocs_arr','deptype',  'Block_Time','Service_Type']]
# y_df

In [11]:
# %%time
# zuludf =  testpd.loc[(testpd["Text_Comment"]== textCommentList[0] ) & (testpd["aocs_dep"].dt.date==uniqueDates[3])]
# zuludf=zuludf.copy()
# temparray = addlastRowArrDep(zuludf)
# masterArray =[]
# [masterArray.append(zulu)  for zulu in temparray ]
# pd.DataFrame(masterArray, columns = ['Dept_Arp', 'engg_arr', 'Flt_Desg', 'Text_Comment', 'Subfleet','Arvl_Arp',  'aocs_dep','engg_dep', 'aocs_arr','deptype',  'Block_Time','Service_Type'])

In [12]:
# %load_ext line_profiler

In [13]:
# %%timeit
# x_df =paramdf.loc[(paramdf["Text_Comment"]== xray) & (paramdf["aocs_dep"].dt.date==alpha)]

In [14]:
#  %lprun -f addlastRowArrDep addlastRowArrDep(zuludf)
# #  Total time: 0.0045326 s
# # Total time: 0.0025298 s
# # Total time: 0.0020349 s
# # Total time: 0.0011861 s
# # Total time: 0.0011546 s
# # Total time: 0.001126 s

In [15]:
def masterDfMaker(paramdf):

    # list of Comment List which are unique
    textCommentList = paramdf["Text_Comment"].unique()
    # list(array of all days in our data )
    uniqueDates = np.sort(testpd["aocs_dep"].dt.day.unique())

    masterList = []
    for xray in textCommentList:
        for alpha in uniqueDates:
            # filtering dataframe on CommentValue and uniqueDateValue
            x_df = paramdf.loc[(paramdf["Text_Comment"] == xray) &
                               (paramdf["cal_date"] == alpha)]
            # check if dataframe is empty
            if len(x_df) == 0:
                continue
            # sorting by aocs_dep for safety
            x_df = x_df.sort_values(by=["aocs_dep"], ascending=True)
            x_df = addlastRowArrDep(x_df)
            [masterList.append(zulu) for zulu in x_df]
    return masterList


# TODO line with .loc is eating 91 percent of the time need to be evaluated

# Wall time: 3min 44s
# Wall time: 5min 35s month April
# Wall time: 6min 6s
# Compiler : 202 ms
# Wall time: 4min 36s
# Last executed at 2021-04-22 09:37:07 in 52.77s now this is achievement

In [16]:
masterList = masterDfMaker(testpd)

In [17]:
%%time
cleanDf = pd.DataFrame(masterList,
                       columns=[
                           'Dept_Arp', 'engg_arr', 'Flt_Desg', 'Text_Comment',
                           'Subfleet', 'Arvl_Arp', 'aocs_dep', 'engg_dep',
                           'aocs_arr', 'deptype', 'Block_Time', 'Service_Type',
                           'cal_date'
                       ])
cleanDf.head(30)
# current time using nparray 50 ms using array
# current time using pythonlist  = Wall time: 175 ms
# Wall time: 493 ms
# 40045 rows × 12 columns using the numpy method
# Wall time: 274 ms old method
# 40044 rows × 12 columns old numpy method

Wall time: 495 ms


,Dept_Arp,engg_arr,Flt_Desg,Text_Comment,Subfleet,Arvl_Arp,aocs_dep,engg_dep,aocs_arr,deptype,Block_Time,Service_Type,cal_date
0,BLR,NaT,6E 6269,A001,6E 32S,MAA,2021-04-01 07:10:00,NaT,2021-04-01 08:15:00,<NA>,0 days 01:05:00,J,1.0
1,MAA,2021-04-01 08:15:00,6E 0991,A001,6E 32S,DEL,2021-04-01 08:50:00,NaT,2021-04-01 11:50:00,<NA>,0 days 03:00:00,J,1.0
2,DEL,2021-04-01 11:50:00,6E 0657,A001,6E 32S,PAT,2021-04-01 12:40:00,NaT,2021-04-01 14:20:00,<NA>,0 days 01:40:00,J,1.0
3,PAT,2021-04-01 14:20:00,6E 0286,A001,6E 32S,DEL,2021-04-01 14:50:00,NaT,2021-04-01 16:25:00,<NA>,0 days 01:35:00,J,1.0
4,DEL,2021-04-01 16:25:00,6E 0286,A001,6E 32S,PNQ,2021-04-01 17:20:00,NaT,2021-04-01 19:15:00,<NA>,0 days 01:55:00,J,1.0
5,PNQ,2021-04-01 19:15:00,6E 0103,A001,6E 32S,BLR,2021-04-01 19:50:00,NaT,2021-04-01 21:30:00,<NA>,0 days 01:40:00,J,1.0
6,BLR,2021-04-01 19:50:00,6E 0103,A001,6E 32S,NaN,NaT,NaT,NaT,NaN,NaT,NaN,NaN
7,BLR,NaT,6E 6269,A001,6E 32S,MAA,2021-04-02 07:10:00,NaT,2021-04-02 08:15:00,<NA>,0 days 01:05:00,J,2.0
8,MAA,2021-04-02 08:15:00,6E 0991,A001,6E 32S,DEL,2021-04-02 08:50:00,NaT,2021-04-02 11:50:00,<NA>,0 days 03:00:00,J,2.0
9,DEL,2021-04-02 11:50:00,6E 0657,A001,6E 32S,PAT,2021-04-02 12:40:00,NaT,2021-04-02 14:20:00,<NA>,0 days 01:40:00,J,2.0


In [18]:
%%time
cleanDf = cleanDf.copy()
# copying data from one column to other
cleanDf["engg_dep"] = cleanDf["aocs_dep"]
# drop column aocs_dep
cleanDf = cleanDf.drop(columns="aocs_dep")
# adding columns cal_date and transit_time
# already added before during refactor
# cleanDf["cal_date"]= np.nan
cleanDf["transit_time"] = np.nan
# avoidy loc error hency copy
cleanDf = cleanDf.copy()
# cal_date calue for base dep/ pd isnull for time is something new
cleanDf.loc[pd.isnull(cleanDf["engg_arr"]),
            "cal_date"] = cleanDf["engg_dep"].dt.date
# cal_date calue for base arr/
cleanDf.loc[pd.isnull(cleanDf["engg_dep"]),
            "cal_date"] = cleanDf["engg_arr"].dt.date
# cal_date calue for transit using the notnull operator
cleanDf.loc[(pd.notnull(cleanDf["engg_arr"]) & pd.notnull(cleanDf["engg_dep"])),
            "cal_date"] = cleanDf["engg_dep"].dt.date
# departuretype as basedep
cleanDf.loc[pd.isnull(cleanDf["engg_arr"]), "deptype"] = "baseDep"
#departure type as baseArr
cleanDf.loc[pd.isnull(cleanDf["engg_dep"]), "deptype"] = "baseArr"
#deparure type as transit
cleanDf.loc[(pd.notnull(cleanDf["engg_arr"]) & pd.notnull(cleanDf["engg_dep"])),
            "deptype"] = "Transit"
# setting type in a column cal_date
cleanDf["cal_date"] = cleanDf["cal_date"].astype("datetime64[ns]")
# calculating transit time
cleanDf.loc[cleanDf['deptype'] == "Transit",
            "transit_time"] = cleanDf["engg_dep"] - cleanDf["engg_arr"]
# setting transitime column type
cleanDf["transit_time"] = cleanDf["transit_time"].astype("timedelta64[ns]")
cleanDf= cleanDf.astype({"deptype":"category","Service_Type":"category","Dept_Arp":"category","Arvl_Arp":"category","Text_Comment":"category","Subfleet":"category","engg_arr":"datetime64[ns]","engg_dep":"datetime64[ns]","Flt_Desg":"category","Block_Time":"timedelta64[ns]"})

# observed improvement of from 4.3 mb to 2.3 mb after setting of category 

Wall time: 871 ms


- [ ] A juypter notebook needs to be tested for two things memory and performance 

In [19]:
cleanDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41993 entries, 0 to 41992
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype          
---  ------        --------------  -----          
 0   Dept_Arp      41993 non-null  category       
 1   engg_arr      35001 non-null  datetime64[ns] 
 2   Flt_Desg      41993 non-null  category       
 3   Text_Comment  41993 non-null  category       
 4   Subfleet      41993 non-null  category       
 5   Arvl_Arp      35643 non-null  category       
 6   engg_dep      35643 non-null  datetime64[ns] 
 7   aocs_arr      35643 non-null  datetime64[ns] 
 8   deptype       41993 non-null  category       
 9   Block_Time    35643 non-null  timedelta64[ns]
 10  Service_Type  35643 non-null  category       
 11  cal_date      41993 non-null  datetime64[ns] 
 12  transit_time  28651 non-null  timedelta64[ns]
dtypes: category(7), datetime64[ns](4), timedelta64[ns](2)
memory usage: 2.3 MB


In [21]:
# pickle_List = [dataframe,days_sorted_list,dep_airport_list]
pickle.dump(cleanDf, open("apr_21.p", "wb"))

gamma = pickle.load(open("apr_21.p", "rb"))

gamma.equals(cleanDf)

True

- comparing the original method and the current method i.e unifieddatatidy vs gsheets

In [ ]:
alpha = pickle.load(open("gsheet_pickle.p", "rb"))

In [ ]:
# testpd.info()
# onedf=testpd.loc[(testpd["Text_Comment"]=="ATR_55")]
# onedf.info()
fourpd = testpd["Text_Comment"].value_counts().reset_index().sort_values(
    'index')
fourpd = fourpd.reset_index(drop=True)
fourpd = fourpd.rename(columns={"Text_Comment": "jebahComment"})

In [ ]:
greta = alpha[0]

threePd = greta["Text Comment"].value_counts().reset_index().sort_values(
    'index')
threePd["Text Comment"] = threePd["Text Comment"].astype(int).floordiv(2)
threePd = threePd.reset_index(drop=True)
threePd = threePd.rename(columns={"Text Comment": "GretaComment"})
#

In [ ]:
fourpd["GretaComment"] = threePd["GretaComment"]
fourpd["reqdiff"] = fourpd["GretaComment"] - fourpd["jebahComment"]
fourpd = fourpd[fourpd.reqdiff != 0].sort_values("reqdiff").reset_index(
    drop=True)
fourpd

- Trying to work with generator 

In [ ]:
testDate = pd.date_range(start='1/1/2021', freq="MS",
                         periods=12).strftime("%b-%y").tolist()
testDate

In [ ]:
cleanDf.loc[:, "engg_dep"].min()